<a href="https://colab.research.google.com/github/cristianbossolasco/scraper-agencias-viajes/blob/main/scraper_busplus_BeautifulSoup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports

In [123]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import re
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Funciones

In [126]:
url = 'https://raw.githubusercontent.com/cristianbossolasco/scraper-agencias-viajes/main/busplus_files/paradas.csv'
df_paradas = pd.read_csv(url, delimiter = ';', encoding = 'unicode_escape')
df_paradas.nombre = df_paradas.nombre.apply(lambda x: x.encode('iso-8859-1').decode('utf-8'))
df_paradas.head()

,id,direccion,latitud,longitud,nombre
0,6,,-35137317600,-6.045238e+10,Bragado - Caminera (Buenos Aires - ARG)
1,7,,-35838385600,-6.187472e+10,Pehuajo - ESSO (Buenos Aires - ARG)
2,8,,-35636855700,-6.135356e+10,Carlos Casares - Shell (Buenos Aires - ARG)
3,10,,-34905308000,-5.795551e+10,La Plata - Terminal (Buenos Aires - ARG)
4,11,,-35451965600,-6.088127e+10,9 de Julio - Terminal (Buenos Aires - ARG)


In [186]:
def get_data_from_page(origen_parada, destino_parada, fecha_ida):
  """
    Realiza el post request a la pagina

    Args:
      origen_parada (str): id de la parade de origen
      destino_parada (str): id de la parada de destino
      fecha_ida (str): fecha de partida en el formato {day}/{month}/{year}

    Returns:
      lista de json con los viajes tal cual los devuelve el servidor
  """

  headers = {
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
  }

  # este response es necesario para obtener una sesion
  response_sesion = requests.post('https://checkout.busplus.com.ar/', headers=headers)
  cookies = {
      'ci_session': response_sesion.cookies['ci_session']
    }

  data = {
      'combinaciones_ida': '0',
      'combinaciones_vuelta': '0',
      'origen_parada': origen_parada,
      'destino_parada': destino_parada,
      'fecha_ida': fecha_ida,
      'fecha_ida_txt': '',
      'fechaFiltroIda': '0',
      'lugarPrecioIda': '0',
      'fecha_vuelta': '',
      'fecha_vuelta_txt': '',
      'fechaFiltroVuelta': '0',
      'lugarPrecioVuelta': '0',
      'solPersona': '1',
      'cupondescuento': '',
  }

  response = requests.post('https://checkout.busplus.com.ar/servicios/obtenerServicios', cookies=cookies, headers=headers, data=data)

  s = BeautifulSoup(response.content, 'html.parser') 
  t = s.find_all('textarea')
  if len(t) == 0:
    return []
  else:
    return json.loads(t[0].text)



def get_data_from_item(json_item):

  ls_categorias = []
  for categoria in json_item['categorias']:

    bEncontro = False
    for i in range(len(ls_categorias)):
      if ls_categorias[i]['descripcion_categoria'] == categoria['descripcion_categoria']:
        bEncontro = True
        break

    if bEncontro:
      ls_categorias[i]['butacas_libres'] += categoria['butacas_libres']
    else:

      json_categoria = {
        'descripcion_categoria': categoria['descripcion_categoria'],
        'importe': categoria['importe'],
        'butacas_libres': categoria['butacas_libres']
      }
      ls_categorias.append(json_categoria)

  if json_item['combinacion_orden'] == None: # Si es un viaje directo

    item_json = {
      "origen_parada": json_item['idorigen'],
      "nombre_origen_parada": json_item['descripcion_origen'],
      "destino_parada": json_item['iddestino'],
      "nombre_destino_parada": json_item['descripcion_destino'],
      "fecha_ida": json_item['fecha_sale'][:10],
      "empresa": json_item['descripcion_web'],
      "salida": json_item['fecha_sale'],
      "llegada": json_item['fecha_llega'],
      "escala": "directo",
      "duracion": json_item['duracion'],
      "categorias": ls_categorias
    }

  else: # Es un viaje con escalas

    item_json = {
      "origen_parada": json_item['idorigen'],
      "nombre_origen_parada": json_item['descripcion_origen'],
      "destino_parada": json_item['iddestino'],
      "nombre_destino_parada": json_item['descripcion_destino'],
      "fecha_ida": fecha_ida,
      "empresa": json_item['descripcion_web'],
      "salida": json_item['fecha_sale'],
      "llegada": json_item['fecha_llega'],
      "escala": "escala",
      "duracion": json_item['duracion'],
      "combinacion": {
        "idcombinacion": json_item['idcombinacion'],
        "combinacion_orden": json_item['combinacion_orden'],
        "combinacion_espera": json_item['combinacion_espera'],
        "combinacion_duracion": json_item['combinacion_duracion'],
      },
      "categorias": ls_categorias
    }


  return item_json

def get_data_directo(json_item):
  ls_categorias = []
  for categoria in json_item['categorias']:

    bEncontro = False
    for i in range(len(ls_categorias)):
      if ls_categorias[i]['descripcion_categoria'] == categoria['descripcion_categoria']:
        bEncontro = True
        break

    if bEncontro:
      ls_categorias[i]['butacas_libres'] += categoria['butacas_libres']
    else:

      json_categoria = {
        'descripcion_categoria': categoria['descripcion_categoria'],
        'importe': categoria['importe'],
        'butacas_libres': categoria['butacas_libres']
      }
      ls_categorias.append(json_categoria)

  item_json = {
    "origen_parada": json_item['idorigen'],
    "nombre_origen_parada": json_item['descripcion_origen'],
    "destino_parada": json_item['iddestino'],
    "nombre_destino_parada": json_item['descripcion_destino'],
    "fecha_ida": json_item['fecha_sale'][:10],
    "empresa": json_item['descripcion_web'],
    "salida": json_item['fecha_sale'],
    "llegada": json_item['fecha_llega'],
    "escala": 'directo',
    "duracion": json_item['duracion'],
    "categorias": ls_categorias
  }

  return item_json  

def get_data(origen_parada, destino_parada, fecha_ida):
  """
    Esta funcion es la encargada de hacer el scraper de un origen y destino puntual en una fecha dada 

    Args:
      origen_parada (str): id de la parade de origen
      destino_parada (str): id de la parada de destino
      fecha_ida (str): fecha de partida en el formato /{day}/{month}/{year}

    Returns:
      lista de json con los viajes
  """

  json_response = get_data_from_page(origen_parada, destino_parada, fecha_ida)
  if json_response == []: return []

  ls_directo = []
  ls_escala = []
  for i in range(len(json_response)):

    item = json_response[i]

    # Si es un viaje directo
    if item['combinacion_orden'] == None:
      ls_directo.append(get_data_directo(json_response[i]))

    # Si es un viaje con escalas
    else:

      item_escala = get_data_from_item(json_response[i])

      if len(ls_escala) == 0:
        ls_escala.append(item_escala)
      else:

        escalas = [{},{},{},{}]

        bEncontro = False
        for i in range(len(ls_escala)):
          if ls_escala[i]['combinacion']['idcombinacion'] == item_escala['combinacion']['idcombinacion']:
            combinacion_orden = ls_escala[i]['combinacion']['combinacion_orden']
            escalas[combinacion_orden - 1] = ls_escala[i]
            bEncontro = True
            break

          if bEncontro:
            combinacion_orden = item_escala['combinacion']['combinacion_orden']
            escalas[combinacion_orden - 1] = item_escala

            categorias = []
            for cat_tramo_1 in escalas[0]["categorias"]:
              for cat_tramo_2 in escalas[1]["categorias"]:
                
                if cat_tramo_1['descripcion_categoria'] == cat_tramo_2['descripcion_categoria']:
                  descripcion_categoria = cat_tramo_1['descripcion_categoria']
                else:
                  descripcion_categoria = cat_tramo_1['descripcion_categoria'] + " | " + cat_tramo_2['descripcion_categoria']

                importe1 = float(cat_tramo_1['importe'])
                importe2 = float(cat_tramo_2['importe'])

                if cat_tramo_1['butacas_libres'] <= cat_tramo_2['butacas_libres']:
                  butacas_libres = cat_tramo_1['butacas_libres']
                else:
                  butacas_libres = cat_tramo_2['butacas_libres']

                json_categoria = {
                  'descripcion_categoria': descripcion_categoria,
                  'importe': importe1 + importe2,
                  'butacas_libres': butacas_libres
                }
                categorias.append(json_categoria)

            item_json = {
              "origen_parada": escalas[0]['idorigen'],
              "nombre_origen_parada": escalas[0]['descripcion_origen'],
              "destino_parada": escalas[1]['iddestino'],
              "nombre_destino_parada": escalas[1]['descripcion_destino'],
              "fecha_ida": fecha_ida,
              "empresa": escalas[0]['descripcion_web'] if escalas[0]['descripcion_web'] == escalas[1]['descripcion_web'] else escalas[0]['descripcion_web'] + ' | ' + escalas[1]['descripcion_web'],
              "salida": escalas[0]['fecha_sale'],
              "llegada": escalas[1]['fecha_llega'],
              "escala": "escala",
              "duracion": escalas[0]['combinacion']['combinacion_duracion'],
              "categorias": categorias
            }


          else:

            ls_escala.append(item_escala)

  return ls_directo


def scrapear_viajes(ls_origenes, ls_destinos, n_dias):
  """
    Busca todas las combinaciones entre los origenes y el destinos, y scrapena n_dias hacia adelante a partir de mañana

    Args:
      ls_origenes (list): lista con los id de origen
      ls_destinos (list): lista con los id de destino
      n_dias (list): cantidad de dias que se va a escrapear a partir de mañana

    Returns:
      lista de json con los viajes
  """

  resultado = []
  contador_origen = 0
  

  for iOrigen in range(len(ls_origenes)):

    id_origen = ls_origenes[iOrigen]
    contador_origen += 1
    contador_destino = 0
    
    for iDestino in range(len(ls_destinos)):

      id_destino = ls_destinos[iDestino]
      contador_destino += 1
      print(f'Origen: idOrigen {id_origen} ({contador_origen} de {len(ls_origenes)}) - Origen: idDestino {id_destino} ({contador_destino} de {len(ls_destinos)})')
      
      if id_destino == id_origen: continue
  
      # incializo con la fecha de mañana
      date = datetime.now() + timedelta(1) 
      for j in range(n_dias): 

          year = str(date.year)
          month = ('00' + str(date.month))[-2:]
          day = ('00' + str(date.day))[-2:]
          fecha = f'{day}/{month}/{year}'
          print(fecha)
          
          result = get_data(id_origen, id_destino, fecha)

          resultado.extend(result)
          date += timedelta(days=1)

  return resultado  

def dict_to_dataframe(result):
  ls = []
  for item in result:
    for categoria in item['categorias']:
      record = {
          'origen_parada': item['origen_parada'], 
          'nombre_origen_parada': item['nombre_origen_parada'], 
          'destino_parada': item['destino_parada'], 
          'nombre_destino_parada': item['nombre_destino_parada'], 
          'fecha_ida': item['fecha_ida'], 
          'empresa': item['empresa'], 
          'salida': item['salida'], 
          'llegada': item['llegada'], 
          'escala': item['escala'], 
          'duracion': item['duracion'], 
          'descripcion_categoria': categoria['descripcion_categoria'],
          'importe': categoria['importe'],
          'butacas_libres': categoria['butacas_libres']
        }
      ls.append(record)
  return pd.DataFrame.from_records(ls)

# scraper

In [ ]:
ls_origenes = [28]
ls_destinos = df_paradas.query('nombre.str.contains("Entre Rios")', engine='python').id.tolist()

result = scrapear_viajes(ls_origenes, ls_destinos, 7)
df_results = dict_to_dataframe(result)
ruta = '/content/drive/MyDrive/viajes/EntreRios_Next7_days.xlsx'
df_results.to_excel(ruta, index=False, encoding = 'utf-8-sig')

Origen: idOrigen 28 (1 de 1) - Origen: idDestino 72 (1 de 20)
28/02/2023
01/03/2023
02/03/2023
03/03/2023
04/03/2023
05/03/2023
06/03/2023
Origen: idOrigen 28 (1 de 1) - Origen: idDestino 98 (2 de 20)
28/02/2023
01/03/2023
02/03/2023
03/03/2023
04/03/2023
05/03/2023
06/03/2023
Origen: idOrigen 28 (1 de 1) - Origen: idDestino 103 (3 de 20)
28/02/2023
01/03/2023
02/03/2023
03/03/2023
04/03/2023
05/03/2023
06/03/2023
Origen: idOrigen 28 (1 de 1) - Origen: idDestino 136 (4 de 20)
28/02/2023
01/03/2023
02/03/2023
03/03/2023
04/03/2023
05/03/2023
06/03/2023
Origen: idOrigen 28 (1 de 1) - Origen: idDestino 159 (5 de 20)
28/02/2023
01/03/2023
02/03/2023
